42 プライベートな属性よりパブリックな属性が好ましい

In [4]:
class MyObject:
    def __init__(self):
        self.public_field = 5
        self.__private_field = 10

    def get_private_field(self):
        return self.__private_field

In [5]:
foo = MyObject()
assert foo.public_field==5

In [6]:
foo.__private_field

AttributeError: 'MyObject' object has no attribute '__private_field'

In [7]:
assert foo.get_private_field()==10

In [8]:
class MyParentObject:
    def __init__(self):
        self.__private_field = 71

class MyChildObject(MyParentObject):
    def get_private_field(self):
        return self.__private_field

baz = MyChildObject()
baz.get_private_field()

AttributeError: 'MyChildObject' object has no attribute '_MyChildObject__private_field'

In [10]:
assert baz._MyParentObject__private_field==71

In [12]:
baz.__dict__

{'_MyParentObject__private_field': 71}

In [13]:
#プライベート属性を使うことを考えるのはサブクラスとの名前の衝突を心配するときのみ
class ApiClass:
    def __init__(self):
        self._value= 5

    def get(self):
        return self._value

class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello'

a= Child()
print(f'{a.get()} and {a._value} should be different')
#衝突してしまう

hello and hello should be different


In [15]:
#衝突を避けるためのプライベート属性
class ApiClass:
    def __init__(self):
        self.__value=5

    def get(self):
        return self.__value

class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello'
a = Child()
print(f'{a.get()} and {a._value} are different')

5 and hello are different


カスタムコンテナ型はcollections.abcを継承する

In [16]:
#要素の頻度を数える追加メソッドを持ったカスタムリスト
class FrequencyList(list):
    def __init__(self, members):
        super().__init__(members)

    def frequency(self):
        counts = {}
        for item in self:
            counts[item] = counts.get(item, 0)+1
        return counts

In [18]:
foo = FrequencyList(['a','b','a','c','b','a','d'])
print('Length',len(foo))
foo.pop()
print('after pop', repr(foo))
print('Frequency:',foo.frequency())

Length 7
after pop ['a', 'b', 'a', 'c', 'b', 'a']
Frequency: {'a': 3, 'b': 2, 'c': 1}


In [49]:
#シーケンスのようにふるまわせるには__getitem__の実装を提供する
class BinaryNode:
    def __init__(self, value, left=None, right= None):
        self.value = value
        self.left = left
        self.right = right

class IndexableNode(BinaryNode):
    def _traverse(self):
        if self.left is not None:
            yield from self.left._traverse()
        yield self
        if self.right is not None:
            yield from self.right._traverse()

    def __getitem__(self, index):
        for i, item in enumerate(self._traverse()):
            if i==index:
                return item.value
        raise IndexError(f'Index {index} is out of range')


In [50]:
#__len__を実装すると長さが返せる
#それでもcountやindexが欠けている
#collections.abcがコンテナ型の典型的なメソッドをすべて提供する抽象基底クラスを定義する

from collections.abc import Sequence

class BadType(Sequence):
    pass

foo = BadType() 

TypeError: Can't instantiate abstract class BadType with abstract methods __getitem__, __len__

In [51]:
class SequenceNode(IndexableNode):
    def __len__(self):
        for count, _ in enumerate(self._traverse(), 1):
            pass
        return count

In [52]:
tree = SequenceNode(
    10,
    left = SequenceNode(
        5,
        left = SequenceNode(2),
        right = SequenceNode(6, right = SequenceNode(7))),
    right= SequenceNode(
        15,
        left = SequenceNode(11)
    )
)

In [53]:
tree[1]

5

In [54]:
print('Tree length:', len(tree))

Tree length: 7


In [55]:
class BetterNode(SequenceNode, Sequence):
    pass

tree = BetterNode(
    10,
    left = BetterNode(
        5,
        left=BetterNode(2),
        right = BetterNode(6,
                            right=BetterNode(7))),
    right= BetterNode(
        15,
        left = BetterNode(11)
    )
)


In [56]:
tree[1]

5

In [57]:
tree.index(7)

3

In [58]:
tree.count(10)

1

44 getメソッドやsetメソッドは使わず属性をそのまま使う

In [62]:
class Resistor:
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0

r1 = Resistor(50e3)
r1.ohms = 10e3

r1.ohms += 10e3

In [63]:
#属性が設定された場合に特別な振る舞いが必要な場合には@propertyデコレータとそれに対応するsetter属性をマイグレーとする

class VoltageResistance(Resistor):
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0
    
r1 = Resistor(50e3)
r1.ohms = 10e3
r1.ohms +=5e3

In [64]:
class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0

    @property
    def voltage(self):
        return self._voltage

    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage/self.ohms

In [65]:
r2 = VoltageResistance(1e3)
print(f'Before: {r2.current:.2f}amps')
r2.voltage = 10
print(f'After: {r2.current:.2f} amps')

Before: 0.00amps
After: 0.01 amps


In [66]:
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)

    @property
    def ohms(self):
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if ohms <=0:
            raise ValueError(f'ohms must be > 0; got {ohms}')
        self._ohms = ohms

In [67]:
r3 = BoundedResistance(1e3)
r3.ohms = 0

ValueError: ohms must be > 0; got 0

In [68]:
BoundedResistance(-5)

ValueError: ohms must be > 0; got -5

In [73]:
#@propertyを使ってスーパークラスの属性を変更不能にする
class FixedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)

    @property
    def ohms(self):
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, '_ohms'):
            raise AttributeError("Can't set attribute")
        self._ohms = ohms

In [74]:
r4 = FixedResistance(1e3)
r4.ohms = 2e3


AttributeError: Can't set attribute

In [75]:
#ゲッタープロパティメソッドの中で他の属性をセットしたりしない

属性をリファクタリングする代わりに@propertyを考える

In [76]:
#水漏れバケツからの水の割り当ての実装

In [78]:
import datetime
from datetime import timedelta
class NewBucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds = period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_cosumed = 0

    def __repr__(self):
        return (f'NewBucket(max_quota={self.max_quota}, '
                f'quota_cosumed = {self.quota_cosumed}')

    @property
    def quota(self):
        return self.max_qquota - self.quota_cosumed

    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount ==0:
            self.quota_consumed = 0
            self.max_quota = 0

        elif delta<0:
            self.max_quota = amount + self.quota_cosumed

        else:
            self.quota_consumed = delta

46 再利用可能な@propertyメソッドにディスクリプタを使う

In [79]:
#@propertyにおける大きな問題は, 再利用デコレータするメソッドを同じクラスの複数の属性で再利用することができない

class Homework:
    def __init__(self):
        self._grade = 0

    @property
    def grade(self):
        return self._grade

    @grade.setter
    def grade(self, value):
        if not (0<=value<=100):
            raise ValueError('grade must be between 0 and 100')
        self._grade = value


In [80]:
galileo = Homework()
galileo.grade = 95

In [81]:
#個々の成績を登録しようとするとそれぞれ@property@setterしなければならない
#Examという新たなクラスをGradeのインスタンスであるクラス属性を持たせるように定義する
#Gradeはディスクリプタプロトコルを実装する
#exam = Exam()
#exam.writing_grade = 40 は
#Exam.__dict__['writing_grade'].__get__(exam, Exam)　と等しい

class Grade:
    def __init__(self):
        self._value = 0

    def __get__(self, instance, instance_type):
        return self._value

    def __set__(self, instance, value):
        if not (0<=value<=100):
            raise ValueError('Grade must be beween 0 and 100')
        self._value = value
        

In [90]:
class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)


Writing 82
Science 99


In [85]:
#複数のExamインスタンスに対してアクセスすると, 予期せぬ振る舞いをする
second_exam = Exam()
second_exam.writing_grade = 75
print(f'Second {second_exam.writing_grade} is right')
print((f'First {first_exam.writing_grade} is wrong, it should be 82'))

#問題は単一のGradeインスタンスが全てのExamインスタンスのクラス属性writing_gradeとして共有されていること

Second 75 is right
First 75 is wrong, it should be 82


In [87]:
#Examインスタンスのそれぞれについて, Gradeクラスで, その値を記録保管する
class Grade:
    def __init_(self):
        self._values = {}

    def __get__(self, instance, instance_type):
        if instance is None:return self
        return self._values.get(instance, 0)

    def __set__(self, instance, value):
        if not(0<=value<=100):
            raise ValueError("Grade must be between 0 and 100")
        self._values[instance]=value

#動くがメモリリークがある

In [88]:
#weakrefを使う
from weakref import WeakKeyDictionary

class Grade:
    def __init__(self):
        self._values = WeakKeyDictionary()

    def __get__(self, instance, instance_type):
        if instance is None:return self
        return self._values.get(instance, 0)

    def __set__(self, instance, value):
        if not(0<=value<=100):
            raise ValueError("Grade must be between 0 and 100")
        self._values[instance]=value        

In [91]:
first_exam = Exam()
first_exam.writing_grade = 82
second_exam = Exam()
second_exam.writing_grade = 75
print(f'First {first_exam.writing_grade} is right')
print(f'Second {second_exam.writing_grade} is right')

First 82 is right
Second 75 is right
